In [14]:
# -*- encoding:utf-8 -*-
#from sklearn import datasets
from numpy import *
import numpy as np
import math
#from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import csv
import os
import heapq
import time
import datetime
from collections import Counter
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from sklearn.model_selection import KFold

starttime=datetime.datetime.now()
import argparse
from sklearn.linear_model import SGDClassifier
Dir=os.path.abspath('')
def Sigmoid(z):
	G_of_Z = float(1.0 / float((1.0 + np.exp(-1.0*z))))
	return G_of_Z

##The hypothesis is the linear combination of all the known factors x[i] and their current estimated coefficients theta[i]
##This hypothesis will be used to calculate each instance of the Cost Function
def Hypothesis(x,theta):
    z = 0
    for i in xrange(0,len(theta)-1):
        z += x[i]*theta[i]
    z=z+theta[-1]
    return z
def loadDataSet(datatype):
    dataMat = []
    labelMat = []
    file='/Feature '+datatype+'.csv'
    Testcsv_file = open(Dir + file, 'r')
    Testcsv_reader = csv.reader(Testcsv_file)
    FeatureList = []  # 会话的特征矩阵
    SessionIDList = []
    LabelList = []  # 会话的Label矩阵
    for data in Testcsv_reader:
        #print data
        SessionID = int(data[0])
        A1 = float(data[1])
        A2 = float(data[2])
        A3 = int(data[3])
        A4 = int(data[4])
        A5 = int(data[5])
        A6 = float(data[6])
        A7 = float(data[7])
        A8 = float(data[8])
        A9 = float(data[9])
        A10 = float(data[10])
        A11 = float(data[11])
        A12 = float(data[12])
        A13 = float(data[13])
        A14 = float(data[14])
        A15 = float(data[15])
        A16 = int(data[16])
        A17 = int(data[17])
        A18 = int(data[18])
        A19 = int(data[19].rstrip(']').lstrip('[').replace(',','').replace(' ',''))#转换为二进制
        A20 = int(data[20].rstrip(']').lstrip('[').replace(',','').replace(' ',''))
        A21 = int(data[21].rstrip(']').lstrip('[').replace(',','').replace(' ',''))
        A22 = float(data[22])
        label = int(data[23])
        User=int(data[24])
        #feature = [A1, A2, A3,A4, 1 / (1 + math.e ** (-random.uniform(0,2)*(0.2*label+0.5))), A6, A7, A8, A9, A10, User, A11, A12, A13, A14, A15, A16, A17, A18, A19, A20, A21,A22]
        #feature = [A1, A2, A3, A4, A5, A9, A10, A11, A12, A13, A14, A15, A16, A17, A18, A19, A20, A21, A22]
        #feature = [A1, A2, A3, A4, A5, A6, A7, A8, A9, A10, User, A11, A12, A13, A14, A15, A16, A17, A18, A19, A20, A21,A22]
        feature = [A1, A2, A3,A4, A5, A6, A7, A8, A9, A10, User, A11, A12, A13, A14, A15, A16, A17, A18, A19, A20, A21,A22]
        dataMat.append(feature)
        #dataMat.append([1, line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10]])
        # dataMat.append([1, float(lineArr[1]), float(lineArr[2]), float(lineArr[3])])
        labelMat.append(label)
    #Unordernum=len(labelMat)

    #print 'Ordering Session:' + str(len(labelMat)-Unordernum)
    dataMat = array(dataMat)
    labelMat = array(labelMat)
    #print dataMat
    #print 'len'+str(len(dataMat[:,0]))
    return dataMat, labelMat
def LRClassifier(data):
    print('\n')
    print("#############*******Co-EM LR **********###############")
    print ('data:' + str(data))
    dataMat, labelMat = loadDataSet(data)
    print('Original dataset shape {}'.format(Counter(labelMat)))
    #plotBestFit(dataMat, labelMat, array(theta))
    #theta = newtonMethod(dataMat, labelMat, 2)
    #K-flod Cross Vlidation#
    X_train, X_test, y_train, y_test = train_test_split(dataMat[:, :], labelMat, test_size=0.9, random_state=None) #初步划分测试集和训练集合
    Plist=[]
    Rlist=[]
    Flist=[]
    #for train_index, test_index in kf.split(X):
    #print("TRAIN:", train_index, "TEST:", test_index)
    #print '\n' + str(f) + '-flod:'
    #f = f + 1
    #X_train, X_test = X[train_index], X[test_index]  # 拆分训练集和测试集
    # X_train, X_test, y_train, y_test = train_test_split(dataMat[:,:], labelMat, test_size=0.3, random_state=None)
    # 为了追求机器学习和最优化算法的最佳性能，我们将特征缩放
    #欠采样
    # rus = RandomUnderSampler(random_state=0)
    # X_train, y_train = rus.fit_sample(X_train, y_train)
    S=11  #对特征分割两个VIEW
    E=23#对特征分割两个VIEW
    sc = StandardScaler()
    sc.fit(X_train)  # 估算每个特征的平均值和标准差
    X_train_std = sc.transform(X_train)
    # 注意：这里我们要用同样的参数来标准化测试集，使得测试集和训练集之间有可比性
    X_test_std = sc.transform(X_test)
    Usertype=X_test[:,S-1]
    #print "UserTpe",Usertype
    FirstUser_std=X_test_std[np.argwhere(Usertype==0)[0][0]][S-1]  #用户类型规格化后数值
    print ("FirstUser_std:",FirstUser_std)
    Evervisit_std=X_test_std[np.argwhere(Usertype==1)[0][0]][S-1]
    print ('Evervisit_std:',Evervisit_std)
    Recent_std=X_test_std[np.argwhere(Usertype==2)[0][0]][S-1]
    print ("Recent_std:",Recent_std)
    X_combined_std = vstack((X_train_std, X_test_std))
    y_combined = hstack((y_train, y_test))
    # 对于训练集进行拆分，标记和未标记2部分数据：
    # 为了追求机器学习和最优化算法的最佳性能，我们将特征规格化
    # print"Start SMOTE Resampling:"
    # sme = SMOTEENN(random_state=42) #neighbor
    # X_train_std, y_train = sme.fit_sample( X_train_std, y_train)

    # print "Start Oversampling:"
    # ros = RandomOverSampler(random_state=0)
    # X_train_std, y_train = ros.fit_sample(X_train_std,y_train)  #Over Sampler
    # print('Oversampling dataset shape {}'.format(Counter(y_train)))
    # print"Start  SMOTE Resampling:"
    # sme = SMOTEENN(random_state=20) #neighbor
    # X_train_std, y_train = sme.fit_sample(X_train_std,y_train)
    # print('SMOTE dataset shape {}'.format(Counter(y_train)))
    D_L=X_train_std
    D_U=X_test_std
    y_L=y_train
    y_U=y_test
    V1_D_Test = array(X_test_std[:, 0:S])
    V2_D_Test = array(X_test_std[:, S:E])
    print ('Scale of D_L data set' + str(Counter(y_L)))
    print ('Scale of D_U data set' + str(Counter(y_U)))
    V1_D_L = array(D_L[:, 0:S])
    V2_D_L = array(D_L[:, S:E])
    V1_D_U = array(D_U[:, 0:S])
    V2_D_U = array(D_U[:, S:E])
    print ('Scale of V1_DL:', np.shape(V1_D_L[:,0:S-1]))
    print ('Scale of V2_DL:', np.shape(V2_D_L))
    # 初始化LR训练计算#
    lr_V2 = LogisticRegression(C=1e5, solver='sag',tol=1e-5,max_iter=5000,multi_class='ovr',class_weight={0: 0.01, 1: 0.01})  # LR参数设置final
    #lr_V2= SGDClassifier(loss="log", penalty="l2",tol=1e-5,max_iter=5000,class_weight={0: 0.01, 1: 0.05})  #SGD梯度下降
    # Co-traing 利用V2上的theta2
    lr_V2.fit(V2_D_L, y_L)  # DL中的V2训练
    # print "lr_V2:",lr_V2
    print ('初始Lr_V2 Coef:' + str(lr_V2.coef_))
    print ('初始Lr_V2 intercept_:' + str(lr_V2.intercept_))
    print ('初始Lr_V2 n_iter_:' + str(lr_V2.n_iter_))
    print ("初始化的LR_V2的预测效果:")
    prediction_V2_Test = lr_V2.predict(V2_D_Test)
    proba_V2_Test = lr_V2.predict_proba(V2_D_Test)
    print(classification_report(y_test, prediction_V2_Test))
    # for i in range(0,len(V2_D_Test)):
    #     feature=V2_D_Test[i].reshape(1, -1)
    #     label=y_test[i]
    #     pro=lr_V2.predict_proba(feature)
    #     print "Label.pro:",label,pro
    print ('Start Co-EM LR:')
    C = 0# 从DU中抽取置信度高的样本加入DL
    Maxinter=20
    print ("从D_U中选择的置信度高的样本量,置信度C:", C)
    #imbalance=50 #抽取的N样本是P样本的倍数
    #print "抽取的Negative样本N和Positive样本数量P比例：",imbalance
    # N=50000
    # P=1000
    # print "选择置信度高的P个D_U+和N个D_U-加入D_L:",'P:',P,'N:',N
    Threshold = 1e-5# 收敛阈值
    print ("迭代收敛条件Threshold:" ,Threshold)# 收敛判决
    print ("最大迭代次数:",Maxinter)
    lambda_u=0.4
    print ("lambda_u:",lambda_u)
    # theta_V1_Last = 0  #收敛判断条件
    # theta_V2_Last = 0#收敛判断条件
    for K in range(0, Maxinter):  # K次迭代
        print ("******************CO-EM LR 第" + str(K + 1) + "次迭代开始:*****************************")
        print ('**************For V=1:*****************')

        print ("Start V1 View:")
        P_init = float(sum(y_L)) / len(y_L)
        print ('Prior P_init:', P_init)
        proba_D_U = lr_V2.predict_proba(V2_D_U)  #DU预测的概率
        #print "logreg_proba",proba_D_U
        proba_D_U = proba_D_U[:, 1].tolist()  # Du在Yes类上概率
        print ("mean of proba_D_U by v2;",np.mean(proba_D_U))
        print ("max of proba_D_U by v2;", np.max(proba_D_U))
        print ("median of proba_D_U by v2",np.median(proba_D_U))
        #proba_DU_No =logreg_proba_D_U[:,0].tolist()  #Du在No类上概率
        #proba_DU_Yes = logreg_proba_D_U  # Du在Yes类上概率
        # print 'proba_DU_Yes:',proba_DU_Yes
        # Co-traing中Positive类的样本个数P和c
        P_U = int(len(y_U) * P_init)  # Positive类的样本个数P,Du+
        N_U = int(len(y_U) - P_U)  # Negative类的样本个数P,Du-
        D_U_P_pro = []  # DU+pro
        D_U_N_pro = []  # DU_pro
        D_U_P_F = []  # DU+_
        D_U_N_F = []  # DU_feature
        #print 'Start:DU中P=1样本DU+:'
        #print 'Start:DU中P=0样本DU-:'
        # starttime1=datetime.datetime.now()
        Dict = {}  # Item:X特征 Value:pro概率值
        for i in range(0, len(y_U)):
            # print 'D_U[i]:',D_U[i]
            feature = D_U[i]
            feature = [str(fe) for fe in feature]  # 转换为字符
            feature = str(feature)
            # print feature
            pro = proba_D_U[i]
            #print "pro:",pro
            Dict.setdefault(feature, pro)
        Dict_sort = sorted(Dict.items(), key=lambda x: x[1], reverse=True)
        for x in Dict_sort[0:P_U]:
            # print x[0]
            # print x[1]
            feature = eval(x[0])
            feature = [float(fe) for fe in feature]  # 转换为字符
            pro = float(x[1])
            D_U_P_pro.append(pro)
            D_U_P_F.append(feature)
            # print "feature:",feature
            # print "pro:",pro
        for x in Dict_sort[P_U:len(y_U)]:
            # print x[0]
            # print x[1]
            feature = eval(x[0])
            feature = [float(fe) for fe in feature]  # 转换为字符
            pro = float(x[1])
            D_U_N_pro.append(pro)
            D_U_N_F.append(feature)
        print ('DU+:', len(D_U_P_pro))
        print ('DU_:', len(D_U_N_pro))
        #print 'Start:DL中P=1样本DL+:'
        #print 'Start:DL中P=0样本DL-:'
        prediction_D_L=lr_V2.predict(V2_D_L)
        proba_D_L=lr_V2.predict_proba(V2_D_L)
        proba_D_L = proba_D_L[:, 1].tolist()  # Du在Yes类上概率
        D_L_P = []  # DL+
        D_L_N = []  # DL_

        P_L = int(sum(y_L))  # Positive类的样本个数P,DL+
        N_L = len(y_L) - P_L  # negative类的样本个数P,DL-
        for i in range(0,len(y_L)):
            if y_L[i]==1:
                D_L_P.append(proba_D_L[i])
            else:
                D_L_N.append(proba_D_L[i])
        print ('DL+:', len(D_L_P))
        print ('DL_:', len(D_L_N))
        print ('Start Estimate Parameter from Dl and Du:')
        u_P = (float(1) / float((lambda_u*len(D_U_P_pro) + len(D_L_P)))) * (sum(D_L_P) + lambda_u*sum(D_U_P_pro))  # U+
        u_N = (float(1) / float((lambda_u*len(D_U_N_pro) + len(D_L_N)))) * (sum(D_L_N) + lambda_u*sum(D_U_N_pro))  # U-
        print ('parameter:u+', u_P)
        print ('parameter:u-', u_N)
        delta_P2List = []
        for x in D_U_P_pro:
            result = (float(1) / float((lambda_u*len(D_U_P_pro) + len(D_L_P)))) * lambda_u*pow(x - u_P, 2)
            delta_P2List.append(float(result))
        for x in D_L_P:
            result = (float(1) / float((lambda_u*len(D_U_P_pro) + len(D_L_P)))) *lambda_u* pow(x - u_P, 2)
            delta_P2List.append(float(result))
        delta_P2 = sum(delta_P2List)  # delta+

        print ('delta_+2:', delta_P2)
        delta_N2List = []
        for x in D_U_N_pro:
            result = (float(1) / float((lambda_u*len(D_U_N_pro) + len(D_L_N))))*lambda_u*pow(x - u_N, 2)
            delta_N2List.append(result)
        for x in D_L_N:
            result = (float(1) / float((lambda_u*len(D_U_N_pro) + len(D_L_N))))*lambda_u*pow(x - u_N, 2)
            delta_N2List.append(result)
        delta_N2 = sum(delta_N2List)  # delta+
        print ('delta_-2:', delta_N2)
        # N_P=1/(float(math.sqrt((delta_P2*2*math.pi))))*math.exp((-1/(2*(float(delta_P2))))*(float(1-u_P))**2)
        # print 'N_P:',N_P  #高斯分布
        # N_N=1/(float(math.sqrt((delta_N2*2*math.pi))))*math.exp((-1/(2*(float(delta_N2))))*(float(0-u_N))**2)
        # print 'N_N:',N_N  #高斯分布
        Pro_1xList = []  # EM得到的H=1的权重
        Pro_0xList = []
        Weight_DU = []  # DU权重
        D_U_S = []  # DU_S选择加入V1的训练集合倒排序
        y_U_S = []  # 对于的label
        Weight_DU_S = []  # 对应的权重
        print ('在V2上，选择置信度高的D_U:')  # 为了降低样本数量和噪音，选取置信度高的D_U,加入D_L
        for I in range(0, len(D_U_P_pro)):
            # print 'x:'+str(x)
            # fx=lr_V2.predict_proba([x])[:,1]
            fx =D_U_P_pro[I]
            # fx=float(fx[0])
            Nor_P = 1 / (float(math.sqrt((delta_P2 * 2 * math.pi)))) * math.exp(
                (-1 / (2 * (float(delta_P2)))) * (float(fx - u_P)) ** 2)
            Nor_N = 1 / (float(math.sqrt((delta_N2 * 2 * math.pi)))) * math.exp(
                (-1 / (2 * (float(delta_N2)))) * (float(fx - u_N)) ** 2)

            Pro_1x = Nor_P * P_init / (Nor_P * P_init + Nor_N * (1 - P_init))  # CO-EM SVM equation(8)
            Pro_0x = 1 - Pro_1x
            Pro_1xList.append(Pro_1x)
            Pro_0xList.append(Pro_0x)
            weight = abs(Pro_1x - Pro_0x)
            Weight_DU.append(weight)
            feature = D_U_P_F[I]
            if weight > C:
                D_U_S.append(feature)
                Weight_DU_S.append(weight)
                if Pro_1x>Pro_0x:
                    y_U_S.append(1)
                else:
                    y_U_S.append(0)
        for I in range(0, len(D_U_N_pro)):
            # print 'x:'+str(x)
            # fx=lr_V2.predict_proba([x])[:,1]
            fx = D_U_N_pro[I]
            # fx=float(fx[0])
            Nor_P = 1 / (float(math.sqrt((delta_P2 * 2 * math.pi)))) * math.exp(
                (-1 / (2 * (float(delta_P2)))) * (float(fx - u_P)) ** 2)
            Nor_N = 1 / (float(math.sqrt((delta_N2 * 2 * math.pi)))) * math.exp(
                (-1 / (2 * (float(delta_N2)))) * (float(fx - u_N)) ** 2)

            Pro_1x = Nor_P * P_init / (Nor_P * P_init + Nor_N * (1 - P_init))  # CO-EM SVM equation(8)
            Pro_0x = 1 - Pro_1x
            Pro_1xList.append(Pro_1x)
            Pro_0xList.append(Pro_0x)
            weight = abs(Pro_1x - Pro_0x)
            #weight=Pro_1x
            Weight_DU.append(weight)
            feature = D_U_N_F[I]
            if  weight>C:
                D_U_S.append(feature)
                Weight_DU_S.append(weight)
                if Pro_1x>Pro_0x:
                    y_U_S.append(1)
                else:
                    y_U_S.append(0)
        # 选择D_U,weight高的若干分类，并在V1上加入D_U,label,weight
        Weight_DU=array(Weight_DU)
        # print "mean of Pro_1_Du;", np.mean(Pro_1xList)
        # print "max of Pro_1_Du;", np.max(Pro_1xList)
        # print "mdedian of Pro_1_Du;",np.median(Pro_1xList)

        D_U_S = array(D_U_S)  #置信度大于C的样本
        Weight_DU_S=array(Weight_DU_S)
        V2_D_U_S = array(D_U_S[:, S:E])
        # print "sharp of V2_D_U_S:", array(V2_D_U_S).shape
        #prediction_D_U_S, logreg_proba_D_U_S = SGDLRPrediction(V2_D_U_S, theta2)  # Sigmod预测结果极其概率
        print("对未标记样本D_U，分类完成，待加入标记样本D_L:")
        #print '初步The scale of y_U_S(加入的D_U):',D_U_S.shape[0]
        print ('The scale of 待加入的D_U:', Counter(y_U_S))
        #采样达到均衡：
        D_U_S_P=[]#正类样本
        D_U_S_N=[]#负类样本
        y_U_S_P=[]
        y_U_S_N=[]
        Weight_DU_S_P=[]
        Weight_DU_S_N=[]
        for i in range(len(y_U_S)):
            if y_U_S[i]==1:
                y_U_S_P.append(1)
                D_U_S_P.append(D_U_S[i])
                Weight_DU_S_P.append(Weight_DU_S[i])
            else:
                y_U_S_N.append(0)
                D_U_S_N.append(D_U_S[i])
                Weight_DU_S_N.append(Weight_DU_S[i])
        #N =  int(len(y_U_S_P)/P_init)# 选取的N样本是P类样本数量
        try:
            print ("mean of Weight_DU_S_N;",np.mean(Weight_DU_S_N))
            print ("mean of Weight_DU_S_P;", np.mean(Weight_DU_S_P))
            print ("median of Weight_DU_S_N;",np.median(Weight_DU_S_N))
            print ("median of Weight_DU_S_P;", np.median(Weight_DU_S_P))
            print ("max of Weight_DU_S_N;",np.max(Weight_DU_S_N))
            print ("max of Weight_DU_S_P;", np.max(Weight_DU_S_P))
            print ("min of Weight_DU_S_N;",np.min(Weight_DU_S_N))
            print ("min of Weight_DU_S_P;", np.min(Weight_DU_S_P))
        except:
            print ("Non-Purchase Label")

        # print 'sharp of D_L:',shape(D_L)
        # print 'sharp of D_U_S:',shape(D_U_S)
        D_new = np.vstack((D_L, D_U_S))
        D_new_V1 = D_new[:, 0:S]
        # print 'sharp of y_L:',shape(y_L)
        # print 'sharp of y_U_S:',shape(y_U_S)
        # print 'y_L:',y_L
        # print 'y_U_S:',array(y_U_S)
        Weight_DL = np.ones(len(D_L), dtype=np.float)
        Weight_new = np.hstack((Weight_DL, Weight_DU_S))
        D_new = np.vstack((D_L, D_U_S))
        D_new_V1 = D_new[:, 0:S]
        # print 'sharp of y_L:',shape(y_L)
        # print 'sharp of y_U_S:',shape(y_U_S)
        # print 'y_L:',y_L
        # print 'y_U_S:',array(y_U_S)
        y_new = np.hstack((y_L, y_U_S))
        print ("The scale of y_new:",Counter(y_new))
        # 部分样本的V1未必全，因为下面进行样本选择，条件为#
        print ('Start Classify on V1 for DL +D_U with weight by LR:')
        #lr_V1 = LogisticRegression(C=1e5, solver='newton-cg', multi_class='ovr', class_weight='balanced')  # LR参数设置
        lr_V1 = LogisticRegression(C=1e5, solver='sag', multi_class='ovr',tol=1e-4,max_iter=10000,class_weight={0: 0.01, 1:0.08})  # LR参数设置final
        #lr_V1 = SGDClassifier(loss="log", penalty="l2", tol=1e-4, max_iter=5000,class_weight={0: 0.01, 1: 0.05})  # SGD梯度下降
        #lr_V1 = LogisticRegression(C=1e5, solver='newton-cg', multi_class='ovr')
        D_new_V1_W=[]
        y_new_V1=[]
        UsertypeList=[]
        #print "D_new_V1.shape[0]:",D_new_V1.shape[0]
        for i in range(0,D_new_V1.shape[0]):#加入权重
            #样本选择，选择user为1和2的用户
            featureList=D_new_V1[i, 0:S]
            Usertype=featureList[S-1]
            #print 'Usertype:',Usertype
            UsertypeList.append(Usertype)
            if round(Usertype, 2) != round(FirstUser_std, 2):  # 只加入Recent-User到V1中
            #if round(Usertype,2)==round(Recent_std,2):#只加入Recent-User到V1中
                X_list=D_new_V1[i,0:S-1]*Weight_new[i]
                #print X_list
                D_new_V1_W.append(X_list)
                y_new_V1.append(y_new[i])
        #print 'Counter(Usertype):',Counter(UsertypeList)
        print ('样本选择后View V1上训练集合:', Counter(y_new_V1))
        lr_V1.fit(D_new_V1_W, y_new_V1)
        #print "lr_V1:",lr_V1
        theta_X_V1=array(lr_V1.coef_[0])
        theta_n_V1=array(lr_V1.intercept_)
        #print theta_X_V1,theta_n_V1
        theta_V1=np.hstack((theta_X_V1,theta_n_V1))

        # for i in range(0,len(CFproList)):
        #     label=y_test_v1[i]
        #     pro=CFproList[i]
        #     if label==1:
        #         print "Label.pro:",label,pro
        # clf = RandomForestClassifier(max_depth=2, random_state=42, max_features='log2')  # 分类器参数设置
        # #clf.fit(train_X_sample, train_y_sample)  # 训练RF
        # clf.fit(D_new_V1_W, y_new_V1)  # 训练RF
        # #print 'RF Creating Finished!'
        # #print "RF feature importances:"
        # print 'Feature Importance by RF:'
        # print(clf.feature_importances_)  # 特征重要性
        # D_L=D_new #新的训练样本 D_L不变
        # y_L=y_new  #新的训练样本
        # V1_D_L = array(D_L[:, 0:S])
        # V2_D_L = array(D_L[:, S:E])
        print ('*******************For V=2:*******************')
        print ("Start V View:")
        P_init = float(sum(y_L)) / len(y_L)
        print ('Prior P_init:', P_init)
        D_U_ER=[]  #ever and recent user 特征矩阵
        y_U_ER=[]
        for i in range(0,D_U.shape[0]):
            featureList=D_U[i, 0:S]
            Usertype=featureList[S-1]
            #print 'Usertype:',Usertype
            if round(Usertype, 2) != round(FirstUser_std, 2):  # 只加入Recent-User到V1中
                feature = D_U[i]  #DU预测的概率
                D_U_ER.append(feature)
            #print "logreg_proba",proba_D_U
        print ("sharp of D_U_ER:",array(D_U_ER).shape[0])
        proba_D_U_ER=lr_V1.predict_proba(array(D_U_ER)[:,0:S-1])
        proba_D_U = proba_D_U_ER[:, 1].tolist()  # Du在Yes类上概率
        print ("mean of proba_D_U by v2;",np.mean(proba_D_U))
        print ("max of proba_D_U by v2;", np.max(proba_D_U))
        print ("median of proba_D_U by v2",np.median(proba_D_U))
        # Co-traing中Positive类的样本个数P和c
        P_U = int(len(proba_D_U) * P_init)  # Positive类的样本个数P,Du+
        N_U = int(len(proba_D_U) - P_U)  # Negative类的样本个数P,Du-
        #print "P_U and N_U:",P_U,N_U
        D_U_P = []  # DU+
        D_U_N = []  # DU_
        D_U_P_F = []  # DU+_
        D_U_N_F = []  # DU_feature
        print ('Start:DU中P=1样本DU+:')
        print ('Start:DU中P=0样本DU-:')
        # starttime1=datetime.datetime.now()
        Dict = {}  # Item:X特征 Value:pro概率值
        for i in range(0, len(proba_D_U)):
            # print 'D_U[i]:',D_U[i]
            feature = D_U_ER[i]
            feature = [str(fe) for fe in feature]  # 转换为字符
            feature = str(feature)
            # print feature
            pro = proba_D_U[i]
            #print "pro:",pro
            Dict.setdefault(feature, pro)
        Dict_sort = sorted(Dict.items(), key=lambda x: x[1], reverse=True)
        for x in Dict_sort[0:P_U]:
            # print x[0]
            # print x[1]
            feature = eval(x[0])
            feature = [float(fe) for fe in feature]  # 转换为字符
            pro = float(x[1])
            D_U_P.append(pro)
            D_U_P_F.append(feature)
            # print "feature:",feature
            # print "pro:",pro
        for x in Dict_sort[P_U:len(proba_D_U)]:
            # print x[0]
            # print x[1]
            feature = eval(x[0])
            feature = [float(fe) for fe in feature]  # 转换为字符
            pro = float(x[1])
            D_U_N.append(pro)
            D_U_N_F.append(feature)

        print ('DU+:', len(D_U_P))
        print ('DU_:', len(D_U_N))
        prediction_D_L=lr_V1.predict(V1_D_L[:,0:S-1])
        proba_D_L=lr_V1.predict_proba(V1_D_L[:,0:S-1])
        proba_D_L = proba_D_L[:, 1].tolist()  # Du在Yes类上概率
        D_L_P = []  # DL+
        D_L_N = []  # DL_
        P_L = int(sum(y_L))  # Positive类的样本个数P,DL+
        N_L = len(y_L) - P_L  # negative类的样本个数P,DL-
        for i in range(0,len(y_L)):
            if y_L[i]==1:
                D_L_P.append(proba_D_L[i])
            else:
                D_L_N.append(proba_D_L[i])
        print ('DL+:', len(D_L_P))
        print ('DL_:', len(D_L_N))
        print ('Start Estimate Parameter from Dl and Du:')
        u_P = (float(1) / float((len(D_U_P) + len(D_L_P)))) * (sum(D_L_P) + sum(D_U_P))  # U+
        u_N = (float(1) / float((len(D_U_N) + len(D_L_N)))) * (sum(D_L_N) + sum(D_U_N))  # U-
        print ('parameter:u+', u_P)
        print ('parameter:u-', u_N)
        delta_P2List = []
        for x in D_U_P:
            result = (float(1) / float((len(D_U_P) + len(D_L_P)))) * pow(x - u_P, 2)
            delta_P2List.append(result)
        for x in D_L_P:
            result = (float(1) / float((len(D_U_P) + len(D_L_P)))) * pow(x - u_P, 2)
            delta_P2List.append(result)
        delta_P2 = sum(delta_P2List)  # delta+

        print ('delta_+2:', delta_P2)
        delta_N2List = []
        for x in D_U_N:
            result = (float(1) / float((len(D_U_N) + len(D_L_N)))) * pow(x - u_N, 2)
            delta_N2List.append(result)
        for x in D_L_N:
            result = (float(1) / float((len(D_U_N) + len(D_L_N)))) * pow(x - u_N, 2)
            delta_N2List.append(result)
        delta_N2 = sum(delta_N2List)  # delta+
        print ('delta_-2:', delta_N2)
        # N_P=1/(float(math.sqrt((delta_P2*2*math.pi))))*math.exp((-1/(2*(float(delta_P2))))*(float(1-u_P))**2)
        # print 'N_P:',N_P  #高斯分布
        # N_N=1/(float(math.sqrt((delta_N2*2*math.pi))))*math.exp((-1/(2*(float(delta_N2))))*(float(0-u_N))**2)
        # print 'N_N:',N_N  #高斯分布
        Pro_1xList = []  # EM得到的H=1的权重
        Pro_0xList = []
        Weight_DU = []  # DU权重
        D_U_S = []  # DU_S选择加入V1的训练集合倒排序
        y_U_S = []  # 对于的label
        Weight_DU_S = []  # 对应的权重
        #print "len(D_U_P_pro)",len(D_U_P_pro)
        #print "len(D_U_P_F)",len(D_U_P_F)
        for I in range(0, len(D_U_P_F)):
            # print 'x:'+str(x)
            # fx=lr_V2.predict_proba([x])[:,1]
            fx = D_U_P[I]
            # fx=float(fx[0])
            Nor_P = 1 / (float(math.sqrt((delta_P2 * 2 * math.pi)))) * math.exp(
                (-1 / (2 * (float(delta_P2)))) * (float(fx - u_P)) ** 2)
            Nor_N = 1 / (float(math.sqrt((delta_N2 * 2 * math.pi)))) * math.exp(
                (-1 / (2 * (float(delta_N2)))) * (float(fx - u_N)) ** 2)

            Pro_1x = Nor_P * P_init / (Nor_P * P_init + Nor_N * (1 - P_init))  # CO-EM SVM equation(8)
            Pro_0x = 1 - Pro_1x
            Pro_1xList.append(Pro_1x)
            Pro_0xList.append(Pro_0x)
            weight = abs(Pro_1x - Pro_0x)
            Weight_DU.append(weight)
            feature =  D_U_P_F[I]
            if weight > C:
                D_U_S.append(feature)
                Weight_DU_S.append(weight)
                if Pro_1x>Pro_0x:
                    y_U_S.append(1)
                else:
                    y_U_S.append(0)
        #print "D_U_N:",len(D_U_N_F)
        for I in range(0, len(D_U_N)):
            # print 'x:'+str(x)
            # fx=lr_V2.predict_proba([x])[:,1]
            fx = D_U_N[I]
            # fx=float(fx[0])
            Nor_P = 1 / (float(math.sqrt((delta_P2 * 2 * math.pi)))) * math.exp(
                (-1 / (2 * (float(delta_P2)))) * (float(fx - u_P)) ** 2)
            Nor_N = 1 / (float(math.sqrt((delta_N2 * 2 * math.pi)))) * math.exp(
                (-1 / (2 * (float(delta_N2)))) * (float(fx - u_N)) ** 2)

            Pro_1x = Nor_P * P_init / (Nor_P * P_init + Nor_N * (1 - P_init))  # CO-EM SVM equation(8)
            Pro_0x = 1 - Pro_1x
            Pro_1xList.append(Pro_1x)
            Pro_0xList.append(Pro_0x)
            weight = abs(Pro_1x - Pro_0x)
            # weight=Pro_1x
            Weight_DU.append(weight)
            feature = D_U_N_F[I]
            if weight > C:
                D_U_S.append(feature)
                Weight_DU_S.append(weight)
                if Pro_1x>Pro_0x:
                    y_U_S.append(1)
                else:
                    y_U_S.append(0)
        # 选择D_U,weight高的若干分类，并在V1上加入D_U,label,weight
        D_U_S = array(D_U_S)
        Weight_DU_S=array(Weight_DU_S)
        # print "sharp of V2_D_U_S:", array(V2_D_U_S).shape
        #prediction_D_U_S, logreg_proba_D_U_S = SGDLRPrediction(V2_D_U_S, theta2)  # Sigmod预测结果极其概率
        print ("对未标记样本D_U，分类完成，待加入标记样本D_L:")
        print ('The scale of y_U_S(加入分类后的D_U):', Counter(y_U_S))
        #采样达到均衡：
        D_U_S_P=[]#正类样本
        D_U_S_N=[]#负类样本
        y_U_S_P=[]
        y_U_S_N=[]
        Weight_DU_S_P=[]
        Weight_DU_S_N=[]
        for i in range(len(y_U_S)):
            if y_U_S[i]==1:
                y_U_S_P.append(1)
                D_U_S_P.append(D_U_S[i])
                Weight_DU_S_P.append(Weight_DU_S[i])
            else:
                y_U_S_N.append(0)
                D_U_S_N.append(D_U_S[i])
                Weight_DU_S_N.append(Weight_DU_S[i])
        try:
            print ("mean of Weight_DU_S_N;",np.mean(Weight_DU_S_N))
            print ("mean of Weight_DU_S_P;", np.mean(Weight_DU_S_P))
            print ("median of Weight_DU_S_N;",np.median(Weight_DU_S_N))
            print ("median of Weight_DU_S_P;", np.median(Weight_DU_S_P))
            print ("max of Weight_DU_S_N;",np.max(Weight_DU_S_N))
            print ("max of Weight_DU_S_P;", np.max(Weight_DU_S_P))
            print ("min of Weight_DU_S_N;",np.min(Weight_DU_S_N))
            print ("min of Weight_DU_S_P;", np.min(Weight_DU_S_P))
        except:
            print ("Non-Purchase Label")
        #print 'sharp of D_L:',shape(D_L)
        #print 'sharp of D_U_S:',shape(D_U_S)
        D_new = np.vstack((D_L, D_U_S))
        D_new_V2 = D_new[:, S:E]
        # print 'sharp of y_L:',shape(y_L)
        # print 'sharp of y_U_S:',shape(y_U_S)
        # print 'y_L:',y_L
        # print 'y_U_S:',array(y_U_S)
        y_new = np.hstack((y_L, y_U_S))
        Weight_DL = np.ones(len(D_L), dtype=np.float)
        Weight_new = np.hstack((Weight_DL, Weight_DU_S))
        print ("The scale of y_new:",Counter(y_new))
        # 部分样本的V1未必全，因为下面进行样本选择，条件为#
        print ('Start Classify on V2 for DL +D_U with weight by LR:')
        #lr_V2 = LogisticRegression(C=1e5, solver='sag', multi_class='ovr')
        lr_V2 = LogisticRegression(C=1e5, solver='sag', tol=1e-5,max_iter=5000,multi_class='ovr',class_weight={0: 0.01, 1: 0.03})  # LR参数设置final
        #lr_V2 = LogisticRegression(C=1e5, solver='sag', tol=1e-5, max_iter=5000, multi_class='ovr',class_weight={0: 0.01, 1: 0.03})  # LR参数设置
        #lr_V2 = SGDClassifier(loss="log", penalty="l2", tol=1e-4, max_iter=5000,class_weight={0: 0.01, 1: 0.04})  # SGD梯度下降
        #lr_V2 = LogisticRegression(C=1e5, solver='sag', multi_class='ovr',class_weight='balanced')  # LR参数设置
        D_new_V2_W=[]
        for i in range(0,D_new_V2.shape[0]):#加入权重
            X_list=D_new_V2[i,:]*Weight_new[i]
            #print X_list
            D_new_V2_W.append(X_list)
        print ('View V2上训练集合:', Counter(y_new))
        lr_V2.fit(D_new_V2_W, y_new)
        theta_X_V2=lr_V2.coef_[0]
        theta_n_V2=lr_V2.intercept_
        theta_V2=np.hstack((theta_X_V2,theta_n_V2))
        #print "lr_V2:",lr_V2

        print ('Lr_V1 Coef:'+str(lr_V1.coef_))
        print ('Lr_V1 intercept_:'+str(lr_V1.intercept_))
        print ('Lr_V1 n_iter_:'+str(lr_V1.n_iter_))
        CFprediction=[]
        CFproList=[]
        y_test_v1=[]
        for i in range(V1_D_Test.shape[0]):
            feature_V1=V1_D_Test[i,:]
            #print "feature_v1",feature_V1
            #print "feayure_V2",feature_V2
            if round(feature_V1[S-1],2)!=round(FirstUser_std,2):#只加入Recent-User到V1中
                feature_V1=array(feature_V1[0:S-1]).reshape(1,-1)
                pro=lr_V1.predict_proba(feature_V1)
                pro=pro[0,1]
                CFprediction.append(int(round(pro)))
                CFproList.append(pro)
                y_test_v1.append(y_test[i])
                #print "Label.pro:", y_test[i], pro
        print ("Test集合V1的预测效果:")
        print(classification_report(y_test_v1, CFprediction))

        print ('Lr_V2 Coef:'+str(lr_V2.coef_))
        print ('Lr_V2 intercept_:'+str(lr_V2.intercept_))
        print ('Lr_V2 n_iter_:'+str(lr_V2.n_iter_))
        prediction_V2_Test=lr_V2.predict(V2_D_Test)
        proba_V2_Test=lr_V2.predict_proba(V2_D_Test)
        print ("Test集合V2的预测效果:")
        print(classification_report(y_test, prediction_V2_Test))
        # D_L=D_new #加入新的样本后的DL训练样本  #D_L不变
        # y_L=y_new  #加入新的样本后的DL训练样本
        # V1_D_L = array(D_L[:, 0:S])
        # V2_D_L = array(D_L[:, S:E])
        # 对测试集合V1,V2进行预测CombineFunction
        CFprediction=[]
        CFproList=[]
        CFprediction_F = []  # CombineFunction
        CFproList_F = []
        y_test_F = []
        CFprediction_E = []  # CombineFunction
        CFproList_E = []
        y_test_E = []
        CFprediction_R = []  # CombineFunction
        CFproList_R = []
        y_test_R = []
        Combine=0 #是否采用联合分类器,1采样,不采用
        if Combine==1:
            for i in range(V1_D_Test.shape[0]):
                feature_V1=V1_D_Test[i,:]
                feature_V2=V2_D_Test[i,:]
                #print "feature_v1",feature_V1
                #print "feayure_V2",feature_V2
                if round(feature_V1[S-1],2)==round(FirstUser_std,2):#初次访问用户,只有f—V2
                    feature_V2=array(feature_V2).reshape(1,-1)
                    pro=lr_V2.predict_proba(feature_V2)
                    pro=pro[0,1]
                    CFprediction_F.append(int(round(pro)))
                    CFproList_F.append(pro)
                    y_test_F.append(y_test[i])
                elif round(feature_V1[S - 1], 2) == round(Evervisit_std, 2):
                    h_V1 = Hypothesis(feature_V1[0:S - 1], theta_V1)
                    h_V2 = Hypothesis(feature_V2, theta_V2)
                    pro = 0.5*(Sigmoid(h_V1- theta_V1[-1])+Sigmoid( h_V2))
                    CFprediction_E.append(int(round(pro)))
                    CFproList_E.append(pro)
                    y_test_E.append(y_test[i])
                elif round(feature_V1[S - 1], 2) == round(Recent_std, 2):
                    feature_V1 = array(feature_V1[0:S-1]).reshape(1, -1)
                    feature_V2 = array(feature_V2).reshape(1,-1)
                    #print "feature_V2",feature_V2
                    pro_V1=lr_V1.predict_proba(feature_V1)
                    pro_V1=pro_V1[0,1]
                    pro_V2=lr_V2.predict_proba(feature_V2)
                    pro_V2 = pro_V2[0,1]
                    pro=0.5*(pro_V1+pro_V2)
                    CFprediction_R.append(int(round(pro)))
                    CFproList_R.append(pro)
                    y_test_R.append(y_test[i])
                #print "pro and label:",round(pro),y_test[i]
                CFprediction.append(int(round(pro)))
                CFproList.append(pro)
        else:
            #分别计算三类用户的性能
            for i in range(V1_D_Test.shape[0]):
                feature_V1 = V1_D_Test[i, :]
                feature_V2 = V2_D_Test[i, :]
                if round(feature_V1[S - 1], 2) == round(FirstUser_std, 2):  # 初次访问用户,只有f—V2
                    h_V2 = Hypothesis(feature_V2, theta_V2)
                    # print 'h_V2:',h_V2
                    pro = Sigmoid(h_V2)
                    # if y_test[i]==1:
                    #     print "feature_v2", feature_V2[0:S - 1]
                    #     print "pro:", pro
                    CFprediction_F.append(int(round(pro)))
                    CFproList_F.append(pro)
                    y_test_F.append(y_test[i])
                elif round(feature_V1[S - 1], 2) == round(Evervisit_std, 2):
                    h_V1 = Hypothesis(feature_V1[0:S - 1], theta_V1)
                    h_V2 = Hypothesis(feature_V2, theta_V2)
                    h_V1_V2 = 1 * (h_V1 + h_V2)
                    pro = Sigmoid(h_V1_V2)
                    #if y_test[i] == 1:
                        # print "feature_v1", feature_V1[0:S - 1]
                        # print "feayure_V2", feature_V2
                        # print "h_V2:",h_V2
                        # print "h_V1,h_V2,h_V1_V2:", h_V1, h_V2, h_V1_V2
                        # print "pro:", pro
                    CFprediction_E.append(int(round(pro)))
                    CFproList_E.append(pro)
                    y_test_E.append(y_test[i])
                elif round(feature_V1[S - 1], 2) == round(Recent_std, 2):
                    # print "feature_v1",feature_V1
                    # print "feayure_V2",feature_V2
                    h_V1 = Hypothesis(feature_V1[0:S - 1], theta_V1)
                    h_V2 = Hypothesis(feature_V2, theta_V2)
                    h_V1_V2 = 1 * (h_V1 + h_V2)
                    # print "h_V1,h_V2,h_V1_V2:",h_V1,h_V2,h_V1_V2
                    # print "pro and label:",round(pro),y_test[i]
                    pro = Sigmoid(h_V1_V2)
                    # print "pro:",pro
                    # pro = Sigmoid(h_V2)
                    # if y_test[i]==1:
                    #     print "feature_v1", feature_V1[0:S - 1]
                    #     print "feayure_V2", feature_V2
                    #     print "h_V1,h_V2,h_V1_V2:",h_V1,h_V2,h_V1_V2
                    #     print "pro and label:",pro,y_test[i]
                    CFprediction_R.append(int(round(pro)))
                    CFproList_R.append(pro)
                    y_test_R.append(y_test[i])
                CFprediction.append(int(round(pro)))
                CFproList.append(pro)
        print ("First-visit User,Ever-visit User和Recent-visit User加入V2性能提升情况:")
        print("Combined Function预测效果 For First_visit Users:")
        print(classification_report(y_test_F, CFprediction_F))
        AUC = metrics.roc_auc_score(y_test_F, CFproList_F)
        print ('AUC:', AUC)
        print('\n')
        print("Combined Function预测效果 For Ever_visit Users:")
        print(classification_report(y_test_E, CFprediction_E))
        AUC = metrics.roc_auc_score(y_test_E, CFproList_E)
        print ('AUC:', AUC)
        print('\n')
        print("Combined Function预测效果 For Recent_visit Users:")
        print(classification_report(y_test_R, CFprediction_R))
        AUC = metrics.roc_auc_score(y_test_R, CFproList_R)
        print ('AUC:', AUC)
        print('\n')
        print("Combined Function预测效果 For All Users:")
        print(classification_report(y_test, CFprediction))
        #print 'CFprediction:',CFprediction
        AUC = metrics.roc_auc_score(y_test, CFproList)
        print ('AUC:',AUC)
        file = '/ProAndLabel_D1.csv'
        csvfile = open(Dir + file, 'wb')
        import codecs
        csvfile.write(codecs.BOM_UTF8)
        writer = csv.writer(csvfile)
        for ID in range(0, len(y_test)):
            #print CFproList[ID], y_test[ID]
            writer.writerow([CFproList[ID], y_test[ID]])
        csvfile.close()
        #print(classification_report(y_test, prediction_V2_Test))
        #print "theta_V1:",theta_V1
        #print "theta_V2:", theta_V2
        if K>0: #判决收敛
            T1=theta_V1-theta_V1_Last
            T2=theta_V2-theta_V2_Last
            print ("max T1:",max(T1))
            print ("max T2:",max(T2))
            T=max(max(T1),max(T2))
            if float(T)<Threshold:
                print ("******************CO-EM LR 第" + str(K + 1) + "次迭代结束并收敛:*****************************")
                break
        theta_V1_Last =theta_V1  # 收敛判断条件
        theta_V2_Last =theta_V2 # 收敛判断条件
        endtime = datetime.datetime.now()
        dwell = (endtime - starttime).seconds
        print ('Till Now Run dwell time:' + str(int(dwell/60)) + 'Minutes')
        print ("******************CO-EM LR 第" + str(K + 1) + "次迭代结束:*****************************")
        print ('\n')
        #print "联合V1和V2效果,Combined Funtions:"
        #prediction_V1_V2_Test = [round(0.5 * (proba_V1_Test[i] + proba_V2_Test[i])) for i in range(len(proba_V2_Test))]
        #print(classification_report(y_test, prediction_V1_V2_Test))

    print ("Lr_V1参数配置：", lr_V1)
    print ("Lr_V2参数配置：", lr_V2)
    print ("******************达到最大迭代次数，CO-EM LR 第" + str(K + 1) + "次迭代结束:*****************************")
    print('\n')
    # print(classification_report(y_test, predictionnew))
    #clf = RandomForestClassifier(max_depth=2, random_state=42, max_features='log2')  # 分类器参数设置
    # clf.fit(train_X_sample, train_y_sample)  # 训练RF
    #clf.fit(X_train_std, y_train)  # 训练RF
    #print 'Feature Importance by RF:'
    #print(clf.feature_importances_)  # 特征重要性
    endtime = datetime.datetime.now()
    dwell = (endtime - starttime).seconds
    print ('Run dwell time:' + str(int(dwell / 60)) + 'Minutes')
LRClassifier('data1')


ModuleNotFoundError: No module named 'scipy.optimize._highs._highs_wrapper'